In [34]:
from web3 import Web3, eth
import pandas as pd
import numpy as np
from datetime import datetime
from hexbytes import HexBytes
import time
from web3.middleware import geth_poa_middleware

# Keep ABIs and recurring functions in "utils" to make this code cleaner
from utils import params
InitializableImmutableAdminUpgradeabilityProxyABI = params()

pd.set_option('display.max_columns', 100)

We are going to bring pool information from the following networks:
- Ethereum Mainnet
- Polygon
- Avalanche
- Optimism

We bring data for both version V2 and version V3 of AAVE as appropriate on each network

For each network/version we bring all pairs that can be deposited

In [3]:
# Save in an Excel file all the AAVE pools that we want to track
aavePools = pd.read_excel('aavePools.xlsx')
aavePools.head(2)

,contract_address,reserve,symbol,decimals,blockchain,aave_version,creation_block
0,0x87870bca3f3fd6335c3f4ce8392d69350b4fa4e2,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,LUSD,18,ethereum,v3,16291127
1,0x87870bca3f3fd6335c3f4ce8392d69350b4fa4e2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,ethereum,v3,16291127


## Get Logs from Network

The code is written once, but we iterate through the different networks and versions, changing the parameters that are necessary

In [9]:
# Use a network node to request the past logs 
provider = Web3.HTTPProvider('https://ethereum.blockpi.network/v1/rpc/public') # CHANGE HERE
w3eth = Web3(provider)
w3eth.isConnected()

#Nodes Providers:
#  https://chainlist.org/

True

In [ ]:
# Each network and version has its own contract, although the ABI is identical for all
aaveContract = aavePools[(aavePools.blockchain == 'ethereum') & (aavePools.aave_version == 'v3')].contract_address.unique()[0] # CHANGE HERE

contract = w3eth.eth.contract(address=Web3.toChecksumAddress(aaveContract), abi=InitializableImmutableAdminUpgradeabilityProxyABI)

In [ ]:
# Select the network and version to query
pools = aavePools[(aavePools.blockchain == 'ethereum') & (aavePools.aave_version == 'v2')] # CHANGE HERE
pools.head(1)

In [ ]:
# Create a Range of Blocks to iterate in the Query
step = 5000 # Query X blocks at a time
blocks = range(pools.creation_block.iloc[0], w3eth.eth.block_number, step) # From first Block of the Contract to now

In [ ]:
# Dataframe to save the results
aaveLogs = pd.DataFrame()

### For "AAVE v3"

This For Loop iterates through each group of blocks, retrieves the logs associated with this contract, looks for the ones that match the detailed topics and saves them as columns

The DF has the txHash as an index and the log data as columns.  

In [ ]:
for b in blocks:
    print(f'Doing Blocks {b} to {b + step}', end = (' ' * 10) + '\r')
    
    # Save to file every 100000 blocks
    if (b - pools.creation_block.iloc[0]) % 100000 == 0:
        aaveLogs.to_csv('aavetemp.csv')
        print(f'{" " * 50} Salved at Block {b}', end = '\r') 
    
    # If it stops, continue from block b
    if b < 0:
        continue
    
    else:
        for a in pools.contract_address:
            filter = w3eth.eth.filter({'fromBlock': b, 'toBlock': b + step, 'address': Web3.toChecksumAddress(a)})
            logs = w3eth.eth.get_filter_logs(filter.filter_id)
            time.sleep(2)
            for log in logs:
                # ReserveDataUpdated 
                if log['topics'][0] == HexBytes('0x804c9b842b2748a22bb64b345453a3de7ca54a6ca45ce00d415894979e22897a'):
                    try:
                        decoded_log = contract.events.ReserveDataUpdated().processLog(log)
                    except:
                        log.topics[1] = HexBytes(log.topics[1])
                        decoded_log = contract.events.Repay().processLog(log)
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'blockNumber'] = decoded_log.blockNumber
                    try:
                        aaveLogs.loc[decoded_log.transactionHash.hex(), 'liquidityRate'] = decoded_log['args'].supplyRate
                    except:
                        aaveLogs.loc[decoded_log.transactionHash.hex(), 'liquidityRate'] = decoded_log['args'].liquidityRate
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'stableBorrowRate'] = decoded_log['args'].stableBorrowRate
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'variableBorrowRate'] = decoded_log['args'].variableBorrowRate

                # Supply 
                elif log['topics'][0] == HexBytes('0x2b627736bca15cd5381dcf80b0bf11fd197d01a037c52b927a881a10fb73ba61'):
                    try:
                        decoded_log = contract.events.Supply().processLog(log)
                    except:
                        log.topics[1] = HexBytes(log.topics[1])
                        log.topics[2] = HexBytes(log.topics[2])
                        decoded_log = contract.events.Repay().processLog(log)
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'event'] = decoded_log.event
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'reserve'] = decoded_log['args'].reserve
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'amount'] = decoded_log['args'].amount
                
                # Withdraw 
                elif log['topics'][0] == HexBytes('0x3115d1449a7b732c986cba18244e897a450f61e1bb8d589cd2e69e6c8924f9f7'):
                    try:
                        decoded_log = contract.events.Withdraw().processLog(log)
                    except:
                        log.topics[1] = HexBytes(log.topics[1])
                        log.topics[2] = HexBytes(log.topics[2])
                        log.topics[3] = HexBytes(log.topics[3])
                        decoded_log = contract.events.Repay().processLog(log)
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'event'] = decoded_log.event
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'reserve'] = decoded_log['args'].reserve
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'amount'] = decoded_log['args'].amount
                
                # Borrow 
                elif log['topics'][0] == HexBytes('0xb3d084820fb1a9decffb176436bd02558d15fac9b0ddfed8c465bc7359d7dce0'):
                    try:
                        decoded_log = contract.events.Borrow().processLog(log)
                    except:
                        log.topics[1] = HexBytes(log.topics[1])
                        log.topics[2] = HexBytes(log.topics[2])
                        decoded_log = contract.events.Repay().processLog(log)
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'event'] = decoded_log.event
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'reserve'] = decoded_log['args'].reserve
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'amount'] = decoded_log['args'].amount
                
                # Repay
                elif log['topics'][0] == HexBytes('0xa534c8dbe71f871f9f3530e97a74601fea17b426cae02e1c5aee42c96c784051'):
                    try:
                        decoded_log = contract.events.Repay().processLog(log)
                    except:
                        log.topics[1] = HexBytes(log.topics[1])
                        log.topics[2] = HexBytes(log.topics[2])
                        log.topics[3] = HexBytes(log.topics[3])
                        decoded_log = contract.events.Repay().processLog(log)
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'event'] = decoded_log.event
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'reserve'] = decoded_log['args'].reserve
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'amount'] = decoded_log['args'].amount

                # FlashLoan
                elif log['topics'][0] == HexBytes('0xefefaba5e921573100900a3ad9cf29f222d995fb3b6045797eaea7521bd8d6f0'):
                    try:
                        decoded_log = contract.events.FlashLoan().processLog(log)
                    except:
                        log.topics[1] = HexBytes(log.topics[1])
                        log.topics[2] = HexBytes(log.topics[2])
                        decoded_log = contract.events.Repay().processLog(log)
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'event'] = decoded_log.event
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'reserve'] = decoded_log['args'].asset
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'amount'] = decoded_log['args'].amount

In [5]:
print(aaveLogs.blockNumber.min())
print(aaveLogs.blockNumber.max())
print(aaveLogs.shape)

16496817.0
16885952.0
(20429, 7)


In [ ]:
# Save Result to file
aaveLogs.to_csv('aave-eth-V3.csv')

### For "AAVE v2"

In [ ]:
for b in blocks:
    print(f'Doing Blocks {b} to {b + step}', end = (' ' * 10) + '\r')
    
    # Save to file every 100000 blocks
    if (b - pools.creation_block.iloc[0]) % 100000 == 0:
        aaveLogs.to_csv('aavetemp.csv')
        print(f'{" " * 50} Salved at Block {b}', end = '\r') 
    
    # If it stops, continue from block b
    if b < 0:
        continue
    
    else:
        for a in pools.contract_address:
            filter = w3eth.eth.filter({'fromBlock': b, 'toBlock': b + step, 'address': Web3.toChecksumAddress(a)})
            logs = w3eth.eth.get_filter_logs(filter.filter_id)
            time.sleep(2)
            for log in logs:
                # ReserveDataUpdated 
                if log['topics'][0] == HexBytes('0x804c9b842b2748a22bb64b345453a3de7ca54a6ca45ce00d415894979e22897a'):
                    try:
                        decoded_log = contract.events.ReserveDataUpdated().processLog(log)
                    except:
                        log.topics[1] = HexBytes(log.topics[1])
                        decoded_log = contract.events.Repay().processLog(log)
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'blockNumber'] = decoded_log.blockNumber
                    try:
                        aaveLogs.loc[decoded_log.transactionHash.hex(), 'liquidityRate'] = decoded_log['args'].supplyRate
                    except:
                        aaveLogs.loc[decoded_log.transactionHash.hex(), 'liquidityRate'] = decoded_log['args'].liquidityRate
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'stableBorrowRate'] = decoded_log['args'].stableBorrowRate
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'variableBorrowRate'] = decoded_log['args'].variableBorrowRate

                # Deposit 
                elif log['topics'][0] == HexBytes('0xde6857219544bb5b7746f48ed30be6386fefc61b2f864cacf559893bf50fd951'):
                    try:
                        decoded_log = contract.events.Deposit().processLog(log)
                    except:
                        log.topics[1] = HexBytes(log.topics[1])
                        log.topics[2] = HexBytes(log.topics[2])
                        decoded_log = contract.events.Repay().processLog(log)
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'event'] = decoded_log.event
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'reserve'] = decoded_log['args'].reserve
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'amount'] = decoded_log['args'].amount
                
                # Withdraw 
                elif log['topics'][0] == HexBytes('0x3115d1449a7b732c986cba18244e897a450f61e1bb8d589cd2e69e6c8924f9f7'):
                    try:
                        decoded_log = contract.events.Withdraw().processLog(log)
                    except:
                        log.topics[1] = HexBytes(log.topics[1])
                        log.topics[2] = HexBytes(log.topics[2])
                        log.topics[3] = HexBytes(log.topics[3])
                        decoded_log = contract.events.Repay().processLog(log)
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'event'] = decoded_log.event
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'reserve'] = decoded_log['args'].reserve
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'amount'] = decoded_log['args'].amount
                
                # Borrow 
                elif log['topics'][0] == HexBytes('0xc6a898309e823ee50bac64e45ca8adba6690e99e7841c45d754e2a38e9019d9b'):
                    try:
                        decoded_log = contract.events.Borrow().processLog(log)
                    except:
                        log.topics[1] = HexBytes(log.topics[1])
                        log.topics[2] = HexBytes(log.topics[2])
                        decoded_log = contract.events.Repay().processLog(log)
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'event'] = decoded_log.event
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'reserve'] = decoded_log['args'].reserve
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'amount'] = decoded_log['args'].amount
                
                # Repay
                elif log['topics'][0] == HexBytes('0x4cdde6e09bb755c9a5589ebaec640bbfedff1362d4b255ebf8339782b9942faa'):
                    try:
                        decoded_log = contract.events.Repay().processLog(log)
                    except:
                        log.topics[1] = HexBytes(log.topics[1])
                        log.topics[2] = HexBytes(log.topics[2])
                        log.topics[3] = HexBytes(log.topics[3])
                        decoded_log = contract.events.Repay().processLog(log)
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'event'] = decoded_log.event
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'reserve'] = decoded_log['args'].reserve
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'amount'] = decoded_log['args'].amount

                # FlashLoan
                elif log['topics'][0] == HexBytes('0xefefaba5e921573100900a3ad9cf29f222d995fb3b6045797eaea7521bd8d6f0'):
                    try:
                        decoded_log = contract.events.FlashLoan().processLog(log)
                    except:
                        log.topics[1] = HexBytes(log.topics[1])
                        log.topics[2] = HexBytes(log.topics[2])
                        decoded_log = contract.events.Repay().processLog(log)
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'event'] = decoded_log.event
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'reserve'] = decoded_log['args'].asset
                    aaveLogs.loc[decoded_log.transactionHash.hex(), 'amount'] = decoded_log['args'].amount

In [ ]:
print(aaveLogs.blockNumber.min())
print(aaveLogs.blockNumber.max())
print(aaveLogs.shape)

16496817.0
16885952.0
(20429, 7)


In [ ]:
(block_number - int(aaveLogs.blockNumber.min())) % (block_interval // 12) 

0

In [ ]:
# Save Result to file
aaveLogs.to_csv('aave-eth-V2.csv')

## Getting BlockTime

web3.py does not currently have a simple way to obtain the blocks timestamp in bulk, and querying them one by one against a node is not viable.

Therefore, an estimated timestamp is calculated based on the query of the real timestamp of a block every x time (one hour in this case)

In [ ]:
# If block data was just  Gathered:
minBlockNumber = int(aaveLogs.blockNumber.min())
maxBlockNumber = int(aaveLogs.blockNumber.max())

-----

In [33]:
# If reading from file:
df = pd.read_csv('aave-avalanche_c-V3.csv')
minBlockNumber = df.blockNumber.min()
maxBlockNumber = df.blockNumber.max()
del df

In [58]:
# Use a network node to request the past logs 
provider = Web3.HTTPProvider('https://api.avax.network/ext/bc/C/rpc')
w3eth = Web3(provider)
# w3eth.middleware_onion.inject(geth_poa_middleware, layer=0) # Only for Polygon/Avalanche
w3eth.isConnected()

#Nodes Providers:
#  https://chainlist.org/
#  https://ethereumnodes.com/

True

*************

Define how often to retrieved a real timestamp and indicate the average length of the block

In [ ]:
reference_blocks = []
completed_blocks = []

block_interval = 3600 * 24 # Block interval (in seconds) to retrieve. 1 hour = 3600 seconds 
block_length = 1 # Approximate number of seconds in a block

In [59]:
# Loop over the block numbers and retrieve the timestamps
for block_number in range(minBlockNumber - 1, maxBlockNumber + 1, round(block_interval // block_length, 0)):
        print(f'Doing Block {block_number}', end = (' ' * 10) + '\r') #  
        timestamp = w3eth.eth.get_block(block_number).timestamp
        reference_blocks.append([block_number, timestamp])
        time.sleep(2)
        

# Iterate over the queried blocks to complete the intermediate ones with the average time in that period
for i in range(len(reference_blocks) - 1):
    block1, timestamp1 = reference_blocks[i]
    block2, timestamp2 = reference_blocks[i+1]

    avg_time_diff = (timestamp2 - timestamp1) / (block2 - block1)

    for j in range(block1+1, block2):
        timestamp = int(timestamp1 + (j - block1) * avg_time_diff)
        completed_blocks.append([j, timestamp])

# Add them to the list
completed_blocks.append(reference_blocks[-1])

completed_blocks = pd.DataFrame(completed_blocks, columns = ("blockNumber", "timestamp"))
completed_blocks['timestamp'] = completed_blocks.timestamp.apply(datetime.utcfromtimestamp)

completed_blocks.sort_values('blockNumber', inplace = True)

completed_blocks.head(2)


,blockNumber,timestamp
0,12027673,2022-03-12 20:09:37
1,12027674,2022-03-12 20:09:39


In [60]:
# Sanity Checks
print(f"Input: {minBlockNumber} - {maxBlockNumber}")
print(f"Done:  {completed_blocks.blockNumber.min()} - {completed_blocks.blockNumber.max()}")
completed_blocks.iloc[completed_blocks.shape[0]//2]

Input: 12027673 - 27825369
Done:  12027673 - 27752472


blockNumber               19890073
timestamp      2022-09-15 06:21:02
Name: 7862309, dtype: object

In [61]:
completed_blocks.to_csv(f'avalanche-blocks-{completed_blocks.blockNumber.min()}-{completed_blocks.blockNumber.max()}.csv', index = False)

______________________